# NYC Taxi fare problem - Assignment 1
***
> ##### __Author__: _Vanshaj Lokwani_
> ##### __SBU ID__: _112025869_

In [2]:
# Importing the tools required
import pandas as pd
import numpy as np
import sklearn as skl
from matplotlib import pyplot as plt
import seaborn as sns

The dataset contains 55M rows and trying to load all of them at once causes the system to hang and intermittently crash. For the initial cleaning and testing phase, using a subset of the data. Once we have an idea of the actual cleaning effort required, we can iteratively perform that on the whole dataset for further processing. 

__For the time being, using first 10M rows as our subset. There is no reason behind this and the number was selected by intuition. Choosing 10M rows gives us enough liberty with the data to identify patterns and the data is small enough to not overload the RAM during pre-processing__

In [4]:
# Getting the dataset
NUM_ROWS = 10000000 # 10M ROWS
train_df = pd.read_csv('../dataset/train.csv', nrows=NUM_ROWS)

Displaying the first 5 rows from the dataframe to get a rough idea of the DF looks like and if it's loaded correctly. 

In [5]:
display(train_df.head())

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [6]:
# Few steps to make our life easier in the long run.
# set the new index as the 'key' coumn. The dataset specifies that the key is unique overall
# the it does not make sense to maintain two different indexes. 
train_df.set_index('key', inplace=True, drop=True) 

#getting the columns in a list. Might be helpful later on. 
columns = list(train_df.columns)
display(columns);display(train_df.head())

['fare_amount',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count']

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
key,,,,,,,
2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
